In [1]:
# Pandas is used for data manipulation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.tree import export_graphviz
import pydot
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
import pickle

In [4]:
# Read in data and display first 5 rows
'''
#import data as df from the excel files 
features = pd.read_excel('ED_Data_MegaTable1.xlsx', sheet_name='ED_Data_MegaTable')
print(features.tail(5))
'''

#import data as df from the excel files 
old_df = pd.read_excel('ED_Data_MegaTable1.xlsx', sheet_name='ED_Data_MegaTable')
    
new_df = pd.read_excel('ED_Data_Part3.xlsx', sheet_name='Table 14')
newer_df = pd.read_excel('ED_Data_Part3.xlsx', sheet_name='Table 15')
    
features = pd.concat([old_df, new_df, newer_df])
print(features.tail(5))

        Acable (m^2)  Bave (T)  Bpeak (T)   Evol (J/m3)       Iop (A)  RRR  \
441726      0.000031       4.1     10.975  5.059524e+07  15814.285714  110   
441727      0.000031       4.1     10.975  8.664021e+07  15814.285714  110   
441728      0.000031       4.1     10.975  1.226852e+08  15814.285714  110   
441729      0.000031       4.1     10.975  1.587302e+08  15814.285714  110   
441730      0.000031       4.1     10.975  5.059524e+07  15814.285714  110   

        Thotspot (K)  Timemarg (s)  fStrand  label  rCuSC  
441726         182.8      0.083567     0.86    NaN   1.27  
441727         231.1      0.063267     0.86    NaN   1.27  
441728         270.8      0.049201     0.86    NaN   1.27  
441729         306.8      0.038043     0.86    NaN   1.27  
441730         178.3      0.087768     0.86    NaN   1.38  


C:\Local\chakrabo\Anaconda\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [16]:
print('The shape of our features is:', features.shape)

The shape of our features is: (1761738, 8)


In [17]:
# Descriptive statistics for each column
features.describe()

AttributeError: 'numpy.ndarray' object has no attribute 'describe'

In [7]:
# One-hot encode the data using pandas get_dummies
features = pd.get_dummies(features)

In [8]:
# Labels are the values we want to predict
labels = np.array(features['label'])

# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop(['Thotspot (K)', 'Timemarg (s)', 'label'], axis = 1)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

In [9]:
# Split the data into training and testing sets
#X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [10]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1321303, 8)
Training Labels Shape: (1321303,)
Testing Features Shape: (440435, 8)
Testing Labels Shape: (440435,)


In [12]:
#run the Random Forest Classifier
clf = RandomForestClassifier(n_estimators=1, max_depth=1, criterion = 'entropy', random_state=42)

#Fit the model
clf.fit(train_features, train_labels);

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [20]:
# Use the forest's predict method on the test data
predictions = clf.predict(test_features)

In [ ]:
# Pull out one tree from the forest
tree = clf.estimators_[5]

# Export the image to a dot file
export_graphviz(tree, out_file='tree.dot', 
                feature_names = feature_list,
                class_names=['below_200','200_to_250', '250_to_300', '300_to_350', 'above_350', 'err'],
                rounded = True,
                filled=True,
                precision = 1)

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

# Write graph to a png file
graph.write_png('tree.png')

In [16]:
# Limit depth of tree to 3 levels
rf_small = RandomForestClassifier(n_estimators=50, max_depth = 5)
rf_small.fit(train_features, train_labels)

# Extract the small tree
tree_small = rf_small.estimators_[5]

# Save the tree as a png image
export_graphviz(tree_small, out_file='small_tree1.dot', 
                feature_names = feature_list,
                class_names=['below_200','200_to_250', '250_to_300', '300_to_350', 'above_350', 'err'],
                rounded = True,
                filled=True,
                precision = 1)

(graph, ) = pydot.graph_from_dot_file('small_tree1.dot')

graph.write_png('small_tree1.png');

In [21]:
pred = pd.DataFrame(predictions, columns=['predictions']).to_csv('result.csv')

In [22]:
filename = 'quench_RandomForest_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [15]:
#print("Accuracy:",metrics.accuracy_score(test_labels, predictions))